# Create alarm failure catergory

In [1]:
#import data and adjust data types
import pandas as pd
import os

In [2]:
# Define path to directory and file name
directory = r"C:\Users\agwsi\CF\6. Advanced Analytics and Dashboard design\Project folder\3. Prepared Data"
filename = "fire_incidents_response_columns.csv" 

# Create the full path using os.path.join
path = os.path.join(directory, filename)

# Import data and parse dates directly with dayfirst=True
tfire = pd.read_csv(
    path,
    parse_dates=[
        'date_of_ext_agent_defer', 'date_of_last_TFS_clear', 'date_of_fire_under_control',
        'date_of_TFS_alarm', 'date_of_TFS_arrival'
    ],
    dtype={'cost_dollars': 'int64'},
    index_col=False,  
    dayfirst=True  
)


In [3]:
time_columns = [col for col in tfire.columns if col.startswith('time')]

for col in time_columns:
    tfire[col] = pd.to_datetime(tfire[col], format='%H:%M:%S').dt.time

In [4]:
# enable full view of all columns
pd.set_option('display.max_columns', None)


In [5]:
tfire.head()

,incident_station_area,incident_ward,latitude,longitude,area_of_origin,extent_of_fire,first_material_ignited,ignition_source,possible_cause,alarm_presence,alarm_operation,alarm_failure,alarm_type,alarm_impact_on_evac,date_of_TFS_alarm,time_of_TFS_alarm,date_of_TFS_arrival,time_of_TFS_arrival,date_of_ext_agent_defer,time_of_ext_agent_defer,date_of_fire_under_control,time_of_fire_under_control,date_of_last_TFS_clear,time_of_last_TFS_clear,fire_staus_at_arrival,control_method,business_impact,cost_dollars,persons_displaced,civilian_casualties,persons_rescued,response_time_seconds,response_category
0,114,18,43.760357,-79.412479,Porch or Balcony,Confined to object of origin,Undetermined (formerly 98),Undetermined,Undetermined,Fire alarm system present,Fire alarm system did not operate,Not applicable: Alarm operated OR presence/ope...,Hardwired (standalone),Not applicable: Occupant(s) first alerted by o...,2018-08-24,16:49:36,2018-08-24,16:54:09,2018-08-24,17:06:26,2018-08-24,17:06:28,2018-08-24,17:42:43,Fire with smoke showing only - including vehic...,Extinguished by fire department,No business interruption,3000,0,0,86,273,Average
1,142,7,43.754542,-79.530419,Cooking Area or Kitchen,Confined to part of room/area of origin,Plastic,"Other Cooking Items (eg Toaster, Kettle, elec ...",Under Investigation,Fire alarm system present,Fire alarm system operated,Not applicable: Alarm operated OR presence/ope...,Interconnected,Not applicable: Occupant(s) first alerted by o...,2018-11-24,07:09:12,2018-11-24,07:14:23,2018-11-24,07:19:00,2018-11-24,07:19:00,2018-11-24,12:45:21,Fire extinguished prior to arrival,Extinguished by occupant,Undetermined,50000,28,0,28,311,Average
2,325,27,43.657300,-79.373460,"Living Area (e.g. living, TV, recreation, etc)","Spread beyond room of origin, same floor",Undetermined (formerly 98),Undetermined,Undetermined,Fire alarm system present,Fire alarm system operated,Not applicable: Alarm operated OR presence/ope...,Hardwired (standalone),Some persons (at risk) evacuated as a result o...,2017-02-09,17:45:07,2017-02-09,17:48:49,2017-02-09,18:02:13,2017-02-09,18:36:21,2017-02-09,23:36:33,Flames showing from small area (one storey or ...,Extinguished by fire department,Not applicable (not a business),1000000,130,1,16,222,Quick
3,332,20,43.649350,-79.393700,Undetermined (formerly 98),Entire Structure,Undetermined (formerly 98),Undetermined,Undetermined,Undetermined,Fire alarm system operation undetermined,Not applicable: Alarm operated OR presence/ope...,Type undetermined,Undetermined,2012-10-30,00:42:01,2012-10-30,00:44:58,2012-10-30,00:52:04,2012-10-30,01:28:48,2012-11-01,16:30:31,Flames showing from large area (more than one ...,Extinguished by fire department,May not resume operations,1000000,14,0,12,177,Quick
4,442,5,43.697960,-79.511539,"Sleeping Area or Bedroom (inc. patients room, ...","Spread beyond room of origin, same floor",Bedding,Matches or Lighters (unable to distinguish),Suspected Arson,Fire alarm system present,Fire alarm system operated,Not applicable: Alarm operated OR presence/ope...,Interconnected,Some persons (at risk) evacuated as a result o...,2018-07-08,04:08:50,2018-07-08,04:13:54,2018-07-08,04:35:00,2018-07-08,04:38:28,2018-07-08,16:44:07,Fire with no evidence from street,Extinguished by fire department,Not applicable (not a business),125000,2,8,11,304,Average


## Define categories of alarm failure and group accordingly

In [7]:
# Define categories for alarm failure
human_error_reasons = [
    "Dead battery", 
    "Electrical or battery not connected", 
    "Improper use of unit", 
    "No battery", 
    "Tampered with (vandalism)"
]

system_failure_reasons = ["Unit failure"]

alarm_separated_reasons = [
    "Separated from fire (e.g. wall, etc)", 
    "Remote from fire - smoke did not reach alarm"
]

other_undetermined_reasons = [
    "Other reason", 
    "Reason for inoperation undetermined"
]
no_failure = ["Not applicable: Alarm operated OR presence/operation undetermined"
]

# Create a new column 'alarm_failure_category' and initialize it with 'Unknown'
tfire['alarm_failure_category'] = 'Unknown'

# Assign categories based on the conditions
tfire.loc[tfire['alarm_failure'].isin(human_error_reasons), 'alarm_failure_category'] = 'Human Error'
tfire.loc[tfire['alarm_failure'].isin(system_failure_reasons), 'alarm_failure_category'] = 'System Failure'
tfire.loc[tfire['alarm_failure'].isin(alarm_separated_reasons), 'alarm_failure_category'] = 'Alarm Separated from Fire'
tfire.loc[tfire['alarm_failure'].isin(other_undetermined_reasons), 'alarm_failure_category'] = 'Other Undetermined'
tfire.loc[tfire['alarm_failure'].isin(no_failure), 'alarm_failure_category'] = 'n/a (no failure)'


In [8]:
tfire['alarm_failure_category'].value_counts()

alarm_failure_category
n/a (no failure)             7629
Unknown                      1254
Other Undetermined           1193
Alarm Separated from Fire     384
Human Error                   366
System Failure                169
Name: count, dtype: int64

In [27]:
tfire.to_csv(r"C:\Users\agwsi\CF\6. Advanced Analytics and Dashboard design\Project folder\3. Prepared Data\alarm_failure_column.csv")